In [1]:
import pandas as pd
import model_afolu as mafl
import model_circular_economy as mc
import model_ippu as mi
import numpy as np
import os, os.path
import model_attributes as ma
from attribute_table import AttributeTable
import setup_analysis as sa
import support_functions as sf
import importlib
import warnings
importlib.reload(ma)
importlib.reload(sa)


#import model_attributes as mafrom attribute_table import AttributeTable
#importlib.reload(ma)

/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2710: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2446: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for desig, df in df_by_designation:
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:5405: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this w

MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2710: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2446: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for desig, df in df_by_designation:
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:5405: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this w

MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


<module 'setup_analysis' from '/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/setup_analysis.py'>

In [ ]:
# NOTE:
#   REPLACE get_variables_by_sector WITH 
#   model_attributes.decompose_
#

In [2]:
model_ce = mc.CircularEconomy(sa.model_attributes)
model_afolu = mafl.AFOLU(sa.model_attributes)
model_ippu = mi.IPPU(sa.model_attributes)



/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2575: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cols = list(set(attr_table.table.columns & set(fields_to_filter_on)))
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2575: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cols = list(set(attr_table.table.columns & set(fields_to_filter_on)))
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2575: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cols = list(set(a

In [52]:
#df_all_vars_ippu = pd.DataFrame({"variable": sa.model_attributes.build_varlist("IPPU", variable_type = "input")})
#df_all_vars_ippu["subsector"] = "IPPU"
#df_cur = pd.read_excel(sa.excel_template_path("IPPU", "arg", "demo"))
#df_all_vars_ippu = pd.merge(df_all_vars_ippu, df_cur, how = "left")
#df_all_vars_ippu = df_all_vars_ippu[df_cur.columns].drop_duplicates().sort_values(by = ["subsector", "variable"]).reset_index(drop = True)
#df_all_vars_ippu.to_csv("/Users/jsyme/Desktop/tmp.csv", index = None, encoding = "UTF-8")



In [59]:
# build AFOLU and SOCIOECONOMIC templates
if True:
    # get the
    tp = max(sa.model_attributes.get_time_periods()[0])
    f0, f1 = (f"min_{tp}", f"max_{tp}")
    df_template_afolu = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "tmp_afolu_vars_with_mcmc_new.csv"))
    df_template_afolu.drop([f0, f1], axis = 1, inplace = True)

    df_sampling_range_defaults = sa.model_attributes.build_default_sampling_range_df()

    df_template_afolu = pd.merge(df_template_afolu, df_sampling_range_defaults, how = "left", on = ["variable"])
    df_template_afolu = df_template_afolu.dropna(subset = ["max_35", "min_35"], how = "any")
    df_template_afolu.drop(["src"], axis = 1, inplace = True)

    df_template_afolu["time_series_id"] = np.zeros(len(df_template_afolu)).astype(int)
    df_template_afolu["strategy_id"] = np.zeros(len(df_template_afolu)).astype(int)
    df_template_afolu["uniform_scaling_q"].iloc[~df_template_afolu["uniform_scaling_q"].isna()] = df_template_afolu["uniform_scaling_q"].iloc[2]

    # order
    tp_max = max(sa.model_attributes.get_time_periods()[0])
    fields_minmax = [f"min_{tp_max}", f"max_{tp_max}"]
    fields_prep = [x for x in df_template_afolu.columns if not x.isnumeric() and (x not in fields_minmax)] + fields_minmax
    fields_times = [x for x in df_template_afolu.columns if (x not in fields_prep)]

    df_template_afolu = df_template_afolu[fields_prep + fields_times]
    subsectors = [sa.model_attributes.dict_model_variable_to_subsector[sa.model_attributes.dict_variables_to_model_variables[x]] for x in df_template_afolu["variable"]]
    df_template_afolu["subsector"] = subsectors

    for sec in ["AFOLU", "Socioeconomic"]:

        vars_keep = sa.model_attributes.get_variables_by_sector(sec, "input")
        df_exp = df_template_afolu[df_template_afolu["variable"].isin(vars_keep)].copy().reset_index(drop = True).drop_duplicates().sort_values(by = ["subsector", "variable"])

        dict_out = {
            "strategy_id-0": df_exp, 
            "strategy_id-1": df_exp.iloc[0:0]
        }
        #sf.dict_to_excel(sa.excel_template_path(sec, "arg", "demo", True), dict_out)
else:
    print("Not running AFOLU and SOCIOECONOMIC")

/Users/jsyme/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:579: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


In [5]:
# build CIRCULAR ECONOMY template
if False:
    ##############################
    #    BUILD WASTE TEMPLATE    #
    ##############################

    sec = "Circular Economy"
    l_vars = sa.model_attributes.get_variables_by_sector(sec, "input")
    l_vars.sort()

    df_base = df_exp.iloc[0:0].copy()
    df_base["variable"] = l_vars
    # add subsectors
    subsectors = [sa.model_attributes.dict_model_variable_to_subsector[sa.model_attributes.dict_variables_to_model_variables[x]] for x in df_base["variable"]]
    df_base["subsector"] = subsectors
    df_base.drop(["min_35", "max_35"], axis = 1, inplace = True)
    df_base = pd.merge(df_base, df_sampling_range_defaults)

    df_safe = pd.read_excel(sa.excel_template_path(sec, "arg", "demo", True).replace("_demo.xlsx", "_demo_safe.xlsx"))
    dft_out = pd.merge(df_base[[x for x in df_base.columns if (x != "0")]], df_safe[["variable", "0"]].dropna(subset = ["0"]).reset_index(drop = True), how = "left")
    dft_out = dft_out[df_template_afolu.columns]

    if True:



        dict_out = {
            "strategy_id-0": dft_out, 
            "strategy_id-1": dft_out.iloc[0:0]
        }
        sf.dict_to_excel(sa.excel_template_path(sec, "arg", "demo", True), dict_out)
else:
    print("Note running CIRCULAR ECONOMY")


Note running CIRCULAR ECONOMY


In [32]:
df_sampling_range_defaults = sa.model_attributes.build_default_sampling_range_df()
df_sampling_range_defaults_ippu = df_sampling_range_defaults[
    df_sampling_range_defaults["variable"].isin(sa.model_attributes.build_varlist("IPPU"))
]
df_sampling_range_defaults_ippu.to_csv("/Users/jsyme/Desktop/ippu_tmp.csv", index = None, encoding = "UTF-8")


df_sampling_range_defaults_ippu



,variable,max_35,min_35
605,ef_ippu_tonne_ch4_per_tonne_production_chemicals,1.0,1.0
606,ef_ippu_tonne_ch4_per_tonne_production_metals,1.0,1.0
607,ef_ippu_tonne_co2_per_mmm_gdp_non_energy_products,1.0,1.0
608,ef_ippu_tonne_co2_per_tonne_production_cement,1.0,1.0
609,ef_ippu_tonne_co2_per_tonne_production_chemicals,1.0,1.0
...,...,...,...
709,vol_ippu_recycled_plastic_m3_ww_per_tonne_prod...,1.0,1.0
710,vol_ippu_recycled_rubber_and_leather_m3_ww_per...,1.0,1.0
711,vol_ippu_recycled_textiles_m3_ww_per_tonne_pro...,1.0,1.0
712,vol_ippu_rubber_and_leather_m3_ww_per_tonne_pr...,1.0,1.0


In [9]:
# build IPPU template
if True:
    df_sampling_range_defaults = sa.model_attributes.build_default_sampling_range_df()
    df_sampling_range_defaults_ippu = df_sampling_range_defaults[
        df_sampling_range_defaults["variable"].isin(sa.model_attributes.build_varlist("IPPU"))
    ]
    
    ##############################
    #    BUILD WASTE TEMPLATE    #
    ##############################

    sec = "IPPU"
    l_vars = sa.model_attributes.get_variables_by_sector(sec, "input")
    l_vars.sort()

    df_base = df_exp.iloc[0:0].copy()
    df_base["variable"] = l_vars
    # add subsectors
    subsectors = [sa.model_attributes.dict_model_variable_to_subsector[sa.model_attributes.dict_variables_to_model_variables[x]] for x in df_base["variable"]]
    df_base["subsector"] = subsectors
    df_base.drop(["min_35", "max_35"], axis = 1, inplace = True)
    df_base = pd.merge(df_base, df_sampling_range_defaults)

    df_safe = pd.read_excel(sa.excel_template_path(sec, "arg", "demo", True).replace("_demo.xlsx", "_demo_safe.xlsx"))
    dft_out = pd.merge(df_base[[x for x in df_base.columns if (x != "0")]], df_safe[["variable", "0"]].dropna(subset = ["0"]).reset_index(drop = True), how = "left")
    dft_out = dft_out[df_template_afolu.columns]

    if True:



        dict_out = {
            "strategy_id-0": dft_out, 
            "strategy_id-1": dft_out.iloc[0:0]
        }
        sf.dict_to_excel(sa.excel_template_path(sec, "arg", "demo", True), dict_out)
else:
    print("Note running IPPU")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/ref/ingestion/parameters_demo/model_input_variables_ip_demo_safe.xlsx'

In [57]:
subsectors = ["Industrial Energy", "Energy Fuels"]#sa.model_attributes.get_sector_subsectors("Energy")
vars_subsectors = set().union(*[set(sa.model_attributes.build_varlist(x)) for x in subsectors])



#subsectors = sa.model_attributes.get_sector_subsectors("Energy")
#vars_subsectors = set().union([set(sa.model_attributes.build_varlist("Energy"))])
df_sampling_range_defaults = sa.model_attributes.build_default_sampling_range_df()
df_sampling_range_defaults_energy = df_sampling_range_defaults[
    df_sampling_range_defaults["variable"].isin(vars_subsectors)
].reset_index(drop = True)
df_sampling_range_defaults_energy.to_csv("/Users/jsyme/Desktop/ener_tmp.csv", index = None, encoding = "UTF-8")




In [66]:
subsectors = ["Industrial Energy", "Energy Fuels"]#sa.model_attributes.get_sector_subsectors("Energy")
vars_subsectors = set().union(*[set(sa.model_attributes.build_varlist(x)) for x in subsectors])
#subsectors = sa.model_attributes.get_sector_subsectors("Energy")
#vars_subsectors = set().union([set(sa.model_attributes.build_varlist("Energy"))])
df_sampling_range_defaults = sa.model_attributes.build_default_sampling_range_df()
df_sampling_range_defaults_energy = df_sampling_range_defaults[
    df_sampling_range_defaults["variable"].isin(vars_subsectors)
].reset_index(drop = True)
#df_sampling_range_defaults_energy.to_csv("/Users/jsyme/Desktop/ener_tmp.csv", index = None, encoding = "UTF-8")
##############################
#    BUILD ENERGY TEMPLATE    #
##############################

sec = "Energy"
l_vars = sa.model_attributes.get_variables_by_sector(sec, "input")
l_vars = list(vars_subsectors)
l_vars.sort()

df_base = df_exp.iloc[0:0].copy()
df_base["variable"] = l_vars
# add subsectors
subsectors = [sa.model_attributes.dict_model_variable_to_subsector[sa.model_attributes.dict_variables_to_model_variables[x]] for x in df_base["variable"]]
df_base["subsector"] = subsectors
df_base.drop(["min_35", "max_35"], axis = 1, inplace = True)
df_base = pd.merge(df_base, df_sampling_range_defaults)

#df_safe = pd.read_excel(sa.excel_template_path(sec, "arg", "demo", True).replace("_demo.xlsx", "_demo_safe.xlsx"))
#dft_out = pd.merge(df_base[[x for x in df_base.columns if (x != "0")]], df_safe[["variable", "0"]].dropna(subset = ["0"]).reset_index(drop = True), how = "left")
dft_out = df_base
dft_out = dft_out[df_template_afolu.columns].sort_values(by = ["subsector", "variable"]).reset_index(drop = True)

if True:

    dict_out = {
        "strategy_id-0": dft_out, 
        "strategy_id-1": dft_out.iloc[0:0]
    }
    sf.dict_to_excel(sa.excel_template_path(sec, "arg", "demo", True), dict_out)
else:
    print("Note running IPPU")



In [73]:
dict_1 = {"a": 34, "b": 4}
dict_2 = {"c": 34, "d": 41}

sf.check_keys(dict_1, ["a", "b"]) if (1 > 0) else sf.check_keys(dict_2, ["a", "b"]);


In [20]:


#sa.model_attributes.dict_attributes["abbreviation_subsector"].table


['Energy Fuels',
 'Energy Storage',
 'Energy Technology',
 'Industrial Energy',
 'Other Energy: Stationary Emissions and Carbon Capture and Sequestration',
 'Transportation']

In [4]:
# expand later to integrate strategy dimension
def build_modvar_input_db_from_templates(
    model_attributes: ma.ModelAttributes,
    sectors: list, 
    region: str, 
    template_type: str,
    repl_missing_with_base: bool = True
) -> pd.DataFrame:
    
    ##  run some checksee
    
    # check region
    region = region.lower()
    if region not in model_attributes.dict_attributes["region"].key_values:
        valid_regions = sf.format_print_list(model_attributes.dict_attributes["region"].key_values)
        raise ValueError(f"Invalid region '{region}' specified. Valid regions are {valid_regions}")
        
    # check sectors
    sectors_drop = [x for x in sectors if (x not in model_attributes.all_sectors)]
    if len(sectors_drop) > 0:
        secs_drop = sf.format_print_list(sectors_drop)
        raise ValueError(f"Invalid sectors {secs_drop} found. Valid sectors are {model_attributes.all_sectors}.")
    
    
    ##
    ##  TEMP: 0 only
    ##
    strat_base = 0
    sheet_base = f"{model_attributes.dim_strategy_id}-{strat_base}"
    
    strats_eval = [strat_base]
    
    df_out = []
    
        
    for sec in sectors:
        fp_templ = sa.excel_template_path(sec, region, template_type, True)
        if not os.path.exists(fp_templ):
            raise ValueError(f"Error: path '{fp_templ}' to template not found.")
        # check available sheets and ensure baseline is available
        sheets_avail = pd.ExcelFile(fp_templ).sheet_names
        if sheet_base not in sheets_avail:
             raise ValueError(f"Baseline strategy sheet {sheet_base} not found in '{fp_templ}'. The template must have a sheet for the baseline strategy.")
        
        
        for strat in strats_eval:
            sheet = f"{model_attributes.dim_strategy_id}-{strat}"
            if not sheet in sheets_avail:
                msg = f"Sheet {sheet} not found in '{fp_templ}'. Check the template."
                if repl_missing_with_base:
                    warnings.warn(f"{msg}. The baseline strategy will be used.")
                    sheet = sheet_base
                else:
                    raise ValueError(msg)
            
            # 
            df_tmp = pd.read_excel(fp_templ, sheet_name = sheet)
            df_tmp[model_attributes.dim_strategy_id] = strat
            
            new_cols = [str(x) for x in df_tmp.columns]#[str(x) if not str(x).isnumeric() else int(x) for x in df_tmp.columns]
            dict_rnm = dict(zip(list(df_tmp.columns), new_cols))
            df_tmp.rename(columns = dict_rnm, inplace = True)
            #print(df_tmp.columns)
            
            #
            #   ADD CHECKS FOR TIME PERIODS
            #
            
            
            #
            #   ADD DIFFERENT STEPS FOR NON-BASELINE STRATEGY
            #
            
            if len(df_out) == 0:
                df_out.append(df_tmp)
            else:
                df_out.append(df_tmp[df_out[0].columns])
                
    df_out = pd.concat(df_out, axis = 0).sort_values(by = ["subsector", "variable"]).reset_index(drop = True)
        
    return df_out
        
# function to convert a model variable input database into a simple projection input dataframe
def build_basic_df_from_modvar_inputs(
    model_attributes: ma.ModelAttributes, 
    df_mv: pd.DataFrame
) -> pd.DataFrame:
    """
        build_basic_df_from_modvar_inputs will take a model input database and transform it (under baseline assumptions) into a projection dataframe.

        - model_attributes: a ModelAttributes class

        - df_mv: a dataframe of model variables
    """

    df_mv_out = df_mv[[str(x) for x in model_attributes.get_time_periods()[0]]].transpose().reset_index(drop = True)
    var_fields = list(df_mv["variable"])
    df_mv_out.rename(columns = dict(zip([(x) for x in range(len(df_mv_out.columns))], var_fields)), inplace = True)

    df_mv_out[model_attributes.dim_time_period] = list(range(len(df_mv_out)))
    var_fields.sort()

    return df_mv_out[[model_attributes.dim_time_period] + var_fields]

region = "argentina"
# everything
df_mv_tmp = build_modvar_input_db_from_templates(sa.model_attributes, ["Socioeconomic", "Circular Economy", "AFOLU", "IPPU", "Energy"], region, "demo")       
df_mv_out = build_basic_df_from_modvar_inputs(sa.model_attributes, df_mv_tmp)
df_mv_out.to_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"), index = None, encoding = "UTF-8")

# afolu vars
df_mv_tmp = build_modvar_input_db_from_templates(sa.model_attributes, ["Socioeconomic", "AFOLU"], region, "demo")       
df_mv_out = build_basic_df_from_modvar_inputs(sa.model_attributes, df_mv_tmp)
df_mv_out.to_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_afolu.csv"), index = None, encoding = "UTF-8")

# ce vars
df_mv_tmp = build_modvar_input_db_from_templates(sa.model_attributes, ["Socioeconomic", "Circular Economy"], region, "demo")       
df_mv_out = build_basic_df_from_modvar_inputs(sa.model_attributes, df_mv_tmp)
df_mv_out.to_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_circular_economy.csv"), index = None, encoding = "UTF-8")

# ippu
df_mv_tmp = build_modvar_input_db_from_templates(sa.model_attributes, ["Socioeconomic", "IPPU"], region, "demo")       
df_mv_out = build_basic_df_from_modvar_inputs(sa.model_attributes, df_mv_tmp)
df_mv_out.to_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_ippu.csv"), index = None, encoding = "UTF-8")

# energy
df_mv_tmp = build_modvar_input_db_from_templates(sa.model_attributes, ["Socioeconomic", "IPPU", "Energy"], region, "demo")       
df_mv_out = build_basic_df_from_modvar_inputs(sa.model_attributes, df_mv_tmp)
df_mv_out.to_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_energy.csv"), index = None, encoding = "UTF-8")

# socioeconomic vars
df_mv_tmp = build_modvar_input_db_from_templates(sa.model_attributes, ["Socioeconomic"], region, "demo")       
df_mv_out = build_basic_df_from_modvar_inputs(sa.model_attributes, df_mv_tmp)
df_mv_out.to_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_socioeconomic.csv"), index = None, encoding = "UTF-8")


In [5]:
2+2

4

In [99]:

df_mv_tmp = build_modvar_input_db_from_templates(sa.model_attributes, ["Socioeconomic", "AFOLU"], "arg", "demo")   
df_import = pd.read_csv("/Users/jsyme/Downloads/predefined_data_vars_revised.csv")
df_check = df_mv_tmp[df_mv_tmp["variable"].isin(list(df_import["file"]))]
df_check[["subsector", "variable", "min_35", "max_35"]].sort_values(by = ["subsector", "variable"]).to_csv("~/tmp.csv", index = None, encoding = "UTF-8")

In [106]:
[x for x in df_import["file"] if "soilcarbon" in x]

['ef_agrc_soilcarbon_bevs_and_spices_kg_co2_ha',
 'ef_agrc_soilcarbon_cereals_kg_co2_ha',
 'ef_agrc_soilcarbon_fibers_kg_co2_ha',
 'ef_agrc_soilcarbon_fruits_kg_co2_ha',
 'ef_agrc_soilcarbon_herbs_and_other_perennial_crops_kg_co2_ha',
 'ef_agrc_soilcarbon_nuts_kg_co2_ha',
 'ef_agrc_soilcarbon_other_annual_kg_co2_ha',
 'ef_agrc_soilcarbon_other_woody_perennial_kg_co2_ha',
 'ef_agrc_soilcarbon_pulses_kg_co2_ha',
 'ef_agrc_soilcarbon_rice_kg_co2_ha',
 'ef_agrc_soilcarbon_sugar_cane_kg_co2_ha',
 'ef_agrc_soilcarbon_tubers_kg_co2_ha',
 'ef_agrc_soilcarbon_vegetables_and_vines_kg_co2_ha',
 'ef_lndu_soilcarbon_grasslands_gg_co2_ha']

In [ ]:
# random work
df_safe = pd.read_excel(sa.excel_template_path(sec, "arg", "demo", True).replace("_demo.xlsx", "_demo_safe.xlsx"))
#dft = dict_out["strategy_id-0"]
#dft_out = pd.merge(dft[[x for x in dft.columns if (x != "0")]], df_safe[["variable", "0"]].dropna(subset = ["0"]).reset_index(drop = True), how = "left")




In [ ]:
dft = sa.model_attributes.dict_attributes["region"].table.copy()
dft["region"] = [x.upper() for x in dft["region"]]

df_wb_waste = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/waste_management_data/wb_whatawaste_country_level_data_0.csv")
df_wb_waste = pd.merge(df_wb_waste, dft[["region"]], left_on = ["iso3c"], right_on = ["region"], how = "inner")
[x for x in df_wb_waste.columns if "recy" in x]

np.mean(df_wb_waste["waste_treatment_recycling_percent"].dropna())

In [ ]:
arr = np.array(df_wb_waste[["total_msw_total_msw_generated_tons_year", "population_population_number_of_people"]])#.columns
np.dot(arr[:, 0], arr[:, 1])/(sum(arr[:, 1])*sum(arr[:, 0]))
               
sum((arr[:, 1]*(arr[:, 0]/arr[:, 1]))/sum(arr[:, 1]))

ind_waste = np.array(df_wb_waste[[x for x in df_wb_waste.columns if "special_waste_industrial_waste_tons_year" in x]].fillna(0).sum(axis = 1))
gdp = (np.array(df_wb_waste["gdp"])/1000)
np.mean(ind_waste/gdp)


In [ ]:
### importlib.reload(sf)
importlib.reload(ma)
importlib.reload(sa)
sa.excel_template_path("AFOLU", "arg", "demo", True)
